In [2]:
import pika
import sys
import os
from threading import Thread

In [14]:
def sender(host_name):
    print(host_name)
    connection = pika.BlockingConnection(
        pika.ConnectionParameters(host=host_name))
    channel = connection.channel()

    channel.exchange_declare(exchange='logs', exchange_type='fanout')
    while True:
        message = input()
        channel.basic_publish(exchange='logs', routing_key='', body=message)
        print(" [x] Sent %r" % message)
    

    connection.close()
    

In [17]:
# thread = Thread(target = sender, args=(("localhost",)))
thread = Thread(target =sender, args=(("localhost",)))
thread.start()

localhost


Exception in thread Thread-10:
Traceback (most recent call last):
  File "E:\programy\pyton\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\programy\pyton\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-36b6389b886b>", line 9, in sender
    message = input()
EOFError: EOF when reading a line



In [7]:
def main():
    connection = pika.BlockingConnection(
        pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()

    channel.exchange_declare(exchange='logs', exchange_type='fanout')

    result = channel.queue_declare(queue='', exclusive=True)
    queue_name = result.method.queue

    channel.queue_bind(exchange='logs', queue=queue_name)

    def callback(ch, method, properties, body):
        print(" [x] %r" % body.decode())

    print(' [*] Waiting for logs. To exit press CTRL+C')
    channel.basic_consume(
        queue=queue_name, on_message_callback=callback, auto_ack=True)

    channel.start_consuming()


# if __name__ == '__main__':
#     try:
#         main()
#     except KeyboardInterrupt:
#         print('Interrupted')
#         try:
#             sys.exit(0)
#         except SystemExit:
#             os._exit(0)